# Basic Usage

## Climate indicator computations

`xclim` is a library of climate indicators that operate on [xarray](https://docs.xarray.dev/en/stable/) `DataArray` objects.

`xclim` provides two layers of computations, one responsible for computations and units handling (the computation layer, the **indices**), and the other responsible for input health checks and metadata formatting (the CF layer, referring to the Climate and Forecast convention, the **indicators**). Functions from the computation layer are found in `xclim.indices`, while indicator objects from the CF layer are found in *realm* modules (`xclim.atmos`, `xclim.land` and `xclim.seaIce`). Users should always use the indicators, and maybe revert to indices as a last resort if the indicator machinery becomes too heavy for their special edge case.

To use xclim in a project, import both `xclim` and `xarray`.

In [ ]:
from __future__ import annotations

import xarray as xr

import xclim
from xclim import testing

Index calculations are performed by opening a NetCDF-like file, accessing the variable of interest, and calling the index function, which returns a new `xarray.DataArray`.

For this example, we'll first open a demonstration dataset storing surface air temperature and compute the number of growing degree days (the sum of degrees above a certain threshold) at the yearly frequency. The time frequency parameter, here `YS`, is specified using [pandas offset aliases](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases). Note that some offsets might not be supported for non-standard calendars (e.g. 360_day), see the [xarray.cftime_range documentation](https://docs.xarray.dev/en/stable/generated/xarray.cftime_range.html) for details.

In [ ]:
# Normally, we would use xarray to open a dataset, e.g.:
# ds = xr.open_dataset("your_file.nc")

# For this example, let's use a test dataset from xclim:
ds = testing.open_dataset("ERA5/daily_surface_cancities_1990-1993.nc")
ds.tas

In [ ]:
gdd = xclim.atmos.growing_degree_days(tas=ds.tas, thresh="10.0 degC", freq="YS")
gdd

This computation was made using the `growing_degree_days` **indicator**. The same computation could be made through the **index**. You can see how the metadata is alot poorer here.

In [ ]:
gdd = xclim.indices.growing_degree_days(tas=ds.tas, thresh="10.0 degC", freq="YS")
gdd

The call to `xclim.indices.growing_degree_days` first checked that the input variable units were units of temperature, ran the computation, then set the output's units to the appropriate unit (here ``"K d"`` or Kelvin days). As you can see, the **Indicator** returned the same output, but with more metadata, it also performed more checks as explained below.

`growing_degree_days` makes most sense with **daily input**, but could theoretically accept other source frequencies. The computational layer (*`Index`*) assumes that users have checked that the input data has the expected temporal frequency and has no missing values. However, no checks are performed, so the output data could be wrong (which is why it's always safer to use **`Indicator`** objects from the CF layer, as demonstrated in the following section).

Finally, as almost all indices, the function takes a `freq` argument to specify over what time period it is computed. These are called "Offset Aliases" and are the same as the resampling string arguments. Valid arguments are detailed in [pandas docs](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases) (note that aliases involving "business" notions are not supported by `xarray` and thus could raise issues in xclim.

### Units Handling paradigm

Indices are written in order to be flexible as to the sampling frequency and units of the data. You _can_ use `growing_degree_days` on, for example, the 6-hourly data, but the output will then be in degree-hour units (``"K h"``). Moreover, all units, even when untouched by the calculation, will be reformatted into a CF-compliant symbol format. This behaviour was chosen to ensure consistency between all indices.

Very few indices will convert their output to specific units; Rather, it is the dimensionality that will be consistent on output. The [Units Handling](units.ipynb) page goes more into detail on how unit conversion can easily be done.

This doesn't apply to **`Indicators`**. Those will always output data in a specific unit, the one listed in the `Indicators.cf_attrs` metadata dictionary.

### Conventions

As you may have noticed, the `growing_degree_days` function above was not told along which dimension to operate. In xclim, the temporal dimension is _always_ assumed to be named `"time"`. All functions which reduce or compute over that dimension will expect that name. If you ever have another name in your data, you can simply rename it like:

```
ds = ds.rename(T="time")
```

For other names and attributes, xclim tries to follow different sets of conventions. In particular, input data should follow the [CF conventions](http://cfconventions.org/) whenever possible for variable attributes. Variable names are usually the ones used in [CMIP6](https://clipc-services.ceda.ac.uk/dreq/mipVars.html), when they exist.

##  Indicators

**Indices with Health Checks and Metadata Attributes**

Indicator instances from the CF layer are found in modules bearing the name of the computational realm in which its input variables are typically found: `xclim.atmos`, `xclim.land` and `xclim.seaIce`. These objects run sanity checks on the input variables and set output's metadata according to CF-conventions when applicable. Some checks involve:

* Identifying periods where missing data significantly impacts the calculation and omits calculations for those periods. Those are called "missing methods" and are detailed in section [Health checks](../checks.rst).
* Appending process history and maintaining the historical provenance of file metadata.
* Writing [Climate and Forecast Convention](https://cfconventions.org/) compliant metadata based on the variables and indices calculated.

Those modules are best used for producing NetCDF files that will be shared with users. See [Climate Indicators](../indicators.rst) for a list of available indicators.

If we run the `growing_degree_days` indicator over a non-daily dataset, we'll be warned that the input data is not daily. That is, running ```xclim.atmos.growing_degree_days(ds.air, thresh='10.0 degC', freq='MS')``` will fail with a `ValidationError`:

In [ ]:
# Show that data is not at a daily time frequency

ds6h = xr.tutorial.open_dataset("air_temperature")
xr.infer_freq(ds6h.time)

In [ ]:
gdd = xclim.atmos.growing_degree_days(tas=ds6h.air, thresh="10.0 degC", freq="MS")

Resampling to a daily frequency and running the same indicator succeeds, but we will still get warnings from the CF metadata checks.

In [ ]:
daily_ds = ds6h.resample(time="D").mean(keep_attrs=True)
gdd = xclim.atmos.growing_degree_days(daily_ds.air, thresh="10.0 degC", freq="YS")

To suppress the CF validation warnings, we can set xclim to send these warnings to the log instead of raising a warning or an error. We also could set `data_validation='warn'` to be able to run the indicator on non-daily data. These options can be set globally or within a context manager with [set_options](../api.rst#options-submodule).

The missing method which determines if a period should be considered missing or not can be controlled through the `check_missing` option, globally or contextually. The main missing methods also have options that can be modified.

In [ ]:
with xclim.set_options(
    check_missing="pct",
    missing_options={"pct": dict(tolerance=0.1)},
    cf_compliance="log",
):
    # Change the missing method to "percent", instead of the default "any"
    # Set the tolerance to 10%, periods with more than 10% of missing data
    #     in the input will be masked in the ouput.
    gdd = xclim.atmos.growing_degree_days(daily_ds.air, thresh="10.0 degC", freq="MS")

Some indicators also expose time-selection arguments as `**indexer` keywords. This allows to run the index on a subset of the time coordinates, for example only on a specific season, month, or between two dates in every year. It relies on the [select_time](../xclim.core.rst#xclim.core.calendar.select_time) function. Some indicators will simply select the time period and run the calculations, while others will smartly perform the selection at the right time, when the order of operation makes a difference. All will pass the `indexer` kwargs to the missing value handling, ensuring that the missing values _outside_ the valid time period are **not** considered.

The next example computes the annual sum of growing degree days over 10 °C, but only considering days from the 1st of April to the 30th of September.

In [ ]:
with xclim.set_options(cf_compliance="log"):
    gdd = xclim.atmos.growing_degree_days(
        tas=daily_ds.air, thresh="10 degC", freq="YS", date_bounds=("04-01", "09-30")
    )
gdd

Finally, `xclim` also allows us to call indicators using datasets and variable names.

In [ ]:
with xclim.set_options(cf_compliance="log"):
    gdd = xclim.atmos.growing_degree_days(
        tas="air", thresh="10.0 degC", freq="MS", ds=daily_ds
    )

    # variable names default to xclim names, so we can even do this:
    renamed_daily_ds = daily_ds.rename(air="tas")
    gdd = xclim.atmos.growing_degree_days(
        thresh="10.0 degC", freq="MS", ds=renamed_daily_ds
    )

## Graphics

Xclim does not have specific functions to create graphics. However, it is built to ensure that Indices and Indicators always have appropriate axis-related metadata that libraries like [Matplotlib](https://matplotlib.org) depend on to generate detailed and informative graphics.

This graphical functionality is entirely thanks to `xarray`, so the following examples are applicable to generic `xarray.DataArray` objects. For more examples, see the directions suggested by [xarray's plotting documentation](https://docs.xarray.dev/en/stable/user-guide/plotting.html)

The `xarray` plot functions creates an histogram when the `DataArray` has 3 or more dimensions. In previous steps, `xclim` automatically filled the `long_name` and `units` attributes, which `xarray` uses to label the x-axis.

In [ ]:
import matplotlib.pyplot as plt

print("long_name:", gdd.attrs["long_name"])
print("units:", gdd.attrs["units"])

gdd.plot()
plt.suptitle("Summary Statistics Histogram")
plt.show()

When the DataArray only has a `time` dimension, xarray plots a timeseries. In this case, xarray uses the `long_name` and `units` attributes provided by xclim to label the y-axis.

In [ ]:
gdd.isel(lon=20, lat=10).plot()
plt.suptitle("Time Series at a Given Geographical Coordinate")
plt.show()

When the DataArray only has 2 dimensions, xarray plots a heatmap. In this case, xarray uses the `long_name` and `units` attributes provided by xclim to label the colorbar.

In [ ]:
gdd.sel(time="2013-07-01").plot()
plt.suptitle("Spatial Pattern at a Specific Time Period")
plt.show()


### Writing DataArrays and Datasets to disk

To save the data as a new NetCDF, use `to_netcdf`:

In [ ]:
gdd.to_netcdf("monthly_growing_degree_days_data.nc")

<div class="alert alert-info">

It's possible to save Dataset objects to other file formats. For more information see: [xarray's documentation](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html)

</div>
